
#1.INTRODUCCION:
-Titulo: Austin Animal,Centro de adopcion de Mascotas -Ubicacion: Texas, Estados Unidos -Alumno: Facundo Gonzalo Arce -CoderHouse
Data Set utilizado: kagglehub.dataset_download("aaronschlegel/austin-animal-center-shelter-intakes-and-outcomes") Bibliografia: Kaggle
Este notebook tiene como objetivo aplicar técnicas de selección de variables vistas durante la cursada, entrenar un modelo de clasificación
y evaluar su rendimiento, utilizando el dataset mencionado anteriormente.
Tambien, su intencion es predecir el tipo de resultado de los animales ingresados al centro (adopción, traslado, etc.)utilizando técnicas de machine learning.

In [ ]:
#2.CARGA DE DATOS AL INFORME:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

file_path = '/content/drive/MyDrive/ArceFacundo-Comisión74240/Entrega 1/DataSet-ArceFacundo.csv'

df = pd.read_csv(file_path)
df.head()

# Mostramos información básica del dataset
print("Dimensiones del dataset:", df.shape)  # filas x columnas
df.info()  # tipos de datos y nulos

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dimensiones del dataset: (80681, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80681 entries, 0 to 80680
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   age_upon_outcome  80673 non-null  object
 1   animal_id         80680 non-null  object
 2   animal_type       80680 non-null  object
 3   breed             80680 non-null  object
 4   color             80680 non-null  object
 5   date_of_birth     80680 non-null  object
 6   datetime          80680 non-null  object
 7   monthyear         80680 non-null  object
 8   name              56115 non-null  object
 9   outcome_subtype   36893 non-null  object
 10  outcome_type      80666 non-null  object
 11  sex_upon_outcome  80678 non-null  object
dtypes: object(12)
memory usage: 7.4+ MB


In [ ]:
#3.PROCESAMIENTO DE LOS DATOS:
# Solo mantenemos las columnas útiles
df = df[['animal_type', 'sex_upon_outcome', 'age_upon_outcome', 'outcome_type', 'breed', 'color']]

# Eliminamos las filas con valores nulos
df = df.dropna()

# Convertimos 'age_upon_outcome' a número de días
def convert_age(age):  # <--- bloque modificado
    try:
        num, unit = age.split()
        num = int(num)
        if 'day' in unit:
            return num
        elif 'week' in unit:
            return num * 7
        elif 'month' in unit:
            return num * 30
        elif 'year' in unit:
            return num * 365
        else:
            return None
    except:
        return None

df['age_days'] = df['age_upon_outcome'].apply(convert_age)

# Eliminamos la columna original
df = df.drop(columns=['age_upon_outcome'])

# Traducción de los valores de 'outcome_type' al español  <--- MOVIDO ARRIBA
df['outcome_type'] = df['outcome_type'].replace({
    'Adoption': 'Adopción',
    'Died': 'Fallecido',
    'Disposal': 'Eliminado',
    'Euthanasia': 'Eutanasia',
    'Missing': 'Perdido',
    'Relocate': 'Reubicado',
    'Return to Owner': 'Devuelto al Dueño',
    'Rto-Adopt': 'Adopción Condicional',
    'Transfer': 'Transferido'
})

# Codificamos variables categóricas
df_encoded = pd.get_dummies(df.drop(columns=['outcome_type']))
y = df['outcome_type']

In [ ]:
#4. APLICAMOS "Feature Selection"
from sklearn.feature_selection import SelectKBest, chi2

#Usamos SelectKBest con chi2 para clasificación
selector = SelectKBest(score_func=chi2, k=10)
X_new = selector.fit_transform(df_encoded, y)
selected_features = df_encoded.columns[selector.get_support()]
X = df_encoded[selected_features]

In [ ]:
#5. ENTRENAMIENTO DEL MODELO:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# DIVISION DEL DATASET:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# ENTRENAMIENTO:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
#6. EVALUACION:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nMatriz de Confusión:\n", confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación:\n", classification_report(y_test, y_pred))

Accuracy: 0.6503430035540128

Matriz de Confusión:
 [[8727    0 1025    0   14    0    0    0  475]
 [  34    0   19    0    0    0    0    0    2]
 [1958    0 1961    0   24    0    0    0  536]
 [   7    0    1    0   60    0    0    0    6]
 [ 232    0  228    3  960    3    0    0  439]
 [  24    0   22    0   42    0    0    0  126]
 [   7    0    1    0    0    0    0    0    9]
 [   2    0    0    0    3    0    0    0    0]
 [2197    0  829    0  132    1    0    0 4089]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Reporte de Clasificación:
                       precision    recall  f1-score   support

            Adopción       0.66      0.85      0.74     10241
Adopción Condicional       0.00      0.00      0.00        55
   Devuelto al Dueño       0.48      0.44      0.46      4479
           Eliminado       0.00      0.00      0.00        74
           Eutanasia       0.78      0.51      0.62      1865
           Fallecido       0.00      0.00      0.00       214
             Perdido       0.00      0.00      0.00        17
           Reubicado       0.00      0.00      0.00         5
         Transferido       0.72      0.56      0.63      7248

            accuracy                           0.65     24198
           macro avg       0.29      0.26      0.27     24198
        weighted avg       0.64      0.65      0.64     24198



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#7. CONCLUSIONES:
El modelo Random Forest alcanzó una precisión (accuracy) decente para predecir el tipo de resultado
('outcome_type') de los animales.Elegimos quedarnos con las variables que parecían más importantes para no complicar el modelo con datos que no aportaban mucho. El modelo funcionó bastante bien, aunque se podrían probar otras formas o herramientas para ver si se mejora un poco más el resultado,como XGBoost por ej.

#8. REFERENCIAS:
Dataset: https://www.kaggle.com/datasets/aaronschlegel/austin-animal-center-shelter-intakes-and-outcomes
